In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

from setting import *

In [ ]:
path_dict = path(
    NAME,
    ELEMENTS,
    NMF_K,
    WT_HCC_K,
    H_HCC_K,
    UPLOAD_TO_PLOTLY,
)

In [ ]:
if 'feature' in ELEMENTS:
    
    feature_1d_context_matrix = pd.read_table(
        path_dict['feature_context_matrix_file_path'],
        index_col=0,
    )
    
if 'sample' in ELEMENTS:
    
    sample_1d_context_matrix = pd.read_table(
        path_dict['sample_context_matrix_file_path'],
        index_col=0,
    )

In [ ]:
if 'feature' in ELEMENTS and 'sample' in ELEMENTS:
    
    negative_signal_matrix = -feature_1d_context_matrix.clip(upper=0) + -sample_1d_context_matrix.clip(upper=0).T
    
    positive_signal_matrix = feature_1d_context_matrix.clip(lower=0) + sample_1d_context_matrix.clip(lower=0).T
    
elif 'feature' in ELEMENTS:
    
    negative_signal_matrix = -feature_1d_context_matrix.clip(upper=0)
    
    positive_signal_matrix = feature_1d_context_matrix.clip(lower=0)
    
elif 'sample' in ELEMENTS:
    
    negative_signal_matrix = -sample_1d_context_matrix.clip(upper=0).T
    
    positive_signal_matrix = sample_1d_context_matrix.clip(lower=0).T
    
negative_signal_matrix.index = ('(-) {}'.format(feature) for feature in negative_signal_matrix.index)

positive_signal_matrix.index = ('(+) {}'.format(feature) for feature in positive_signal_matrix.index)

In [ ]:
signal_matrix = pd.concat((
    negative_signal_matrix,
    positive_signal_matrix,
))

signal_matrix.to_csv(
    path_dict['signal_matrix_file_path'],
    sep='\t',
)

signal_matrix

In [ ]:
if FEATURES_FILE_PATH is not None:
    
    features = pd.read_table(
        FEATURES_FILE_PATH,
        squeeze=True,
    ).tolist()
    
    print('Read {} features.'.format(len(features)))
    
    negative_signal_matrix = negative_signal_matrix.loc[[index for index in negative_signal_matrix.index if index[4:] in features]]
    
    print(negative_signal_matrix.shape)
    
    positive_signal_matrix = positive_signal_matrix.loc[[index for index in positive_signal_matrix.index if index[4:] in features]]
    
    print(positive_signal_matrix.shape)

In [ ]:
if SELECT_FEATURE_AUTOMATICALLY:
    
    feature_negative_sum = negative_signal_matrix.sum(axis=1)
    
    selected_negative_features = feature_negative_sum.index[feature_negative_sum.std() / 2 < feature_negative_sum].tolist()
    
    feature_positive_sum = positive_signal_matrix.sum(axis=1)
    
    selected_positive_features = feature_positive_sum.index[feature_positive_sum.std() / 2 < feature_positive_sum].tolist()
    
    ccal.plot_points(
        (
            tuple(range(negative_signal_matrix.shape[0])),
            tuple(range(positive_signal_matrix.shape[0])),
        ),
        (
            feature_negative_sum.sort_values(),
            feature_positive_sum.sort_values(),
        ),
        names=(
            'Negative Signal',
            'Positive Signal',
        ),
    )
    
elif N_TOP_FEATURE is not None:
    
    selected_negative_features = ccal.flatten_nested_iterable(negative_signal_matrix.apply(lambda column: column.dropna().sort_values()[-N_TOP_FEATURE:].index))
    
    selected_positive_features = ccal.flatten_nested_iterable(positive_signal_matrix.apply(lambda column: column.dropna().sort_values()[-N_TOP_FEATURE:].index))
    
else:
    
    selected_negative_features = negative_signal_matrix.index.tolist()
    
    selected_positive_features = positive_signal_matrix.index.tolist()
    
print('Selected {} negative and {} positive features.'.format(
    len(selected_negative_features),
    len(selected_positive_features),
))

In [ ]:
if SELECT_CONTEXT == 'negative':
    
    selected_features = selected_negative_features
    
elif SELECT_CONTEXT == 'positive':
    
    selected_features = selected_positive_features
    
elif SELECT_CONTEXT == 'both':
    
    selected_features = selected_negative_features + selected_positive_features
    
selected_signal_matrix = signal_matrix.loc[selected_features]

In [ ]:
selected_signal_matrix = pd.DataFrame(
    ccal.normalize_nd_array(
        selected_signal_matrix.values,
        1,
        '0-1',
        raise_for_bad_value=False,
    ),
    index=selected_signal_matrix.index,
    columns=selected_signal_matrix.columns,
)

selected_signal_matrix.to_csv(
    path_dict['selected_signal_matrix_file_path'],
    sep='\t',
)

selected_signal_matrix